In [2]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



import ta as ta #libreria para extraccion de indicadores tecnicos 
import yfinance as yf #libreria para descargar datos de productos financieros

In [3]:
# Definimos los tickets de los activo
vandguard_sp500 = 'VOO'
vandguard_nasdaq = 'VGT'
vandguard_dowjones = 'VIG'


In [4]:
def extract_clean_ticker(ticker: str, start=None, end=None, interval='1d') -> pd.DataFrame:
    """
    Descarga y limpia datos históricos de un ticker con yfinance.
    
    Args:
        ticker (str): Símbolo del ticker (ej: 'VOO', '^GSPC')
        start (str): Fecha de inicio 'YYYY-MM-DD'
        end (str): Fecha de fin 'YYYY-MM-DD'
        interval (str): Intervalo temporal ('1d', '1wk', etc.)
        
    Returns:
        pd.DataFrame: DataFrame limpio con columnas estandarizadas.
    """
    df = yf.download(
        tickers=ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=True,
        progress=False
    )
    

    if isinstance(df.columns, pd.MultiIndex):             # Si las columnas del DataFrame son un MultiIndex (por ejemplo, tras descargar datos con varias series),
        df.columns = df.columns.droplevel(1)              # se elimina el segundo nivel del MultiIndex para simplificar.
        df.columns = df.columns.get_level_values(0)       # se selecciona solo el primer nivel de nombres de columna para simplificar el DataFrame.
    
    df = df.reset_index()
    df.rename(columns={"Date": "date"}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.columns = [col.lower() for col in df.columns]
    df = df.sort_values('date')                           # Asegurarse de que los datos están ordenados por fecha
    df = df.round(3)                                       #Redondear los valores float a 3 decimales
    
    # Asegurarse de que las columnas necesarias están presentes
    cols_esperadas = {'open', 'high', 'low', 'close', 'volume'}
    if not cols_esperadas.issubset(df.columns):
        raise ValueError(f"Faltan columnas necesarias en {ticker}: {cols_esperadas - set(df.columns)}")
    
    return df

In [7]:
#extraemos los datos para VOO
df_voo = extract_clean_ticker(vandguard_sp500, start='2010-01-01', end='2025-05-01')
#cambiamos la columna date a tipo datetime
df_voo['date'] = pd.to_datetime(df_voo['date'])
#exportamos los datos a un csv en la carpeta data
df_voo.to_csv('../data/VOO.csv', index=False)